# Finite Difference Method

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys

# Add the path to the mlmc module to the python path
sys.path.append("../src")
from contract import *
from fdm import *
from model import BlackScholes

# Nice plot
import plotly.offline as pyo
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pyo.init_notebook_mode(connected=True)

In [2]:
# Model parameters
interest_rate = 0.05
initial_value = 1.0
sigma = 0.2

model = BlackScholes(interest_rate, initial_value, sigma)

# Contract parameters
maturity = 1.0
strike = 1.0

PDE for European options (in Black-Scholes model) with payoff $G(S_T)$:

$$
\begin{cases}
    \partial_t v+\frac{1}{2}\sigma^2 x^2 \partial^2_x v + rx \partial_x v - rv = 0 &\text{in } [0,T)\times[0,x_{max}) \\
    v(T,x) = G(x) &\text{in } [0,x_{max})
\end{cases}
$$

In [14]:
ue_fdm = EuropeanFDM(l=5, m=100)
xmin, xmax = 0, 2 * model.initial_value

fig = make_subplots(rows=1, cols=2, subplot_titles=("Call", "Put"), specs=[[{"type": "surface"}, {"type": "surface"}]])

# Call
ue_call = EuropeanCall(maturity, strike)
boundary_min = 0
boundary_max = xmax - ue_call.strike * np.exp(
    -model.interest_rate * (ue_call.maturity - np.linspace(0, ue_call.maturity, ue_fdm.m))
)
x, t, u = ue_fdm.solve(model, ue_call, xmin, xmax, boundary_min, boundary_max)
x, t = np.meshgrid(t, x)
fig.add_trace(go.Surface(z=u, x=x, y=t, colorscale="turbo", name=f"{ue_call.name()}", showscale=False), row=1, col=1)

# Put
ue_put = EuropeanPut(maturity, strike)
boundary_min = ue_call.strike * np.exp(
    -model.interest_rate * (ue_call.maturity - np.linspace(0, ue_call.maturity, ue_fdm.m))
)
boundary_max = 0
x, t, u = ue_fdm.solve(model, ue_put, xmin, xmax, boundary_min, boundary_max)
x, t = np.meshgrid(t, x)
fig.add_trace(go.Surface(z=u, x=x, y=t, colorscale="turbo", name=f"{ue_put.name()}", showscale=False), row=1, col=2)

fig.update_layout(
    title="",
    scene=dict(xaxis_title="Time", yaxis_title="Stock price", zaxis_title="Option price"),
    scene2=dict(xaxis_title="Time", yaxis_title="Stock price", zaxis_title="Option price"),
    height=600,
)
fig.show()

PDE for Asian options (in Black-Scholes model):

$$
\begin{cases}
    \partial_t v+\frac{1}{2}\sigma^2 x^2 \partial^2_x v + rx \partial_x v + x \partial_a v - rv = 0 \\
    v(T,x,a) = G(a)
\end{cases}
$$